In [3]:
#importing libaries
import pandas as pd
import numpy as np
import requests
print('libraries imported!')

libraries imported!


In [23]:
#from bs4 import BeautifulSoup
import bs4
from bs4 import BeautifulSoup
import requests
import ssl
import csv

In [32]:
# getting data from internet
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_text = requests.get(wiki).text

In [39]:
pip install BeautifulSoup4

     |████████████████████████████████| 5.8MB 6.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [98]:
# using beautiful soup to parse the HTML/XML codes. (using builtin python html parser)
soup = BeautifulSoup(wiki_text, "html.parser")
#print(soup.prettify())

In [123]:
#using soup object, iterate the wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
df = pd.DataFrame(data = data,columns = columns)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [126]:
#Remove Boroughs that are 'Not assigned'
df = df[df['Borough'] != 'Not assigned']
df.head(5)

,Postal code,Borough,Neighborhood
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [127]:
df.shape

(103, 3)

In [130]:
# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods

df = df.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()

#remove duplicates
df = df.drop_duplicates()

#update index to be postcode
#if(df.index.name != 'Postal code'):
    #df = df.set_index('Postal code')
    
df.head()
df.shape

(103, 3)

In [131]:
# If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough
df['Neighborhood'].replace("Not assigned", df["Borough"],inplace=True)


In [133]:
df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [136]:
df["Neighborhood"]= df["Neighborhood"].str.replace("/", ",")


In [146]:
df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [147]:
df.shape

(103, 3)